In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-apr-2022/sample_submission.csv
/kaggle/input/tabular-playground-series-apr-2022/train_labels.csv
/kaggle/input/tabular-playground-series-apr-2022/train.csv
/kaggle/input/tabular-playground-series-apr-2022/test.csv


In [2]:
df = pd.read_csv('../input/tabular-playground-series-apr-2022/train.csv')
df

,sequence,subject,step,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09,sensor_10,sensor_11,sensor_12
0,0,47,0,-0.196291,0.112395,1.000000,0.329204,-1.004660,-0.131638,-0.127505,0.368702,-0.1,-0.963873,-0.985069,0.531893,4.751492
1,0,47,1,-0.447450,0.134454,1.000000,-0.658407,0.162495,0.340314,-0.209472,-0.867176,0.2,-0.301301,0.082733,-0.231481,0.454390
2,0,47,2,0.326893,-0.694328,1.000000,0.330088,0.473678,1.280479,-0.094718,0.535878,1.4,1.002168,0.449221,-0.586420,-4.736147
3,0,47,3,0.523184,0.751050,1.000000,0.976991,-0.563287,-0.720269,0.793260,0.951145,-0.3,-0.995665,-0.434290,1.344650,0.429241
4,0,47,4,0.272025,1.074580,1.000000,-0.136283,0.398579,0.044877,0.560109,-0.541985,-0.9,1.055636,0.812631,0.123457,-0.223359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1558075,25967,327,55,-0.282844,-1.217437,-1.666153,0.586726,-0.930698,-0.451010,-0.651184,0.368702,0.4,0.008671,-0.723536,-0.353909,-0.914749
1558076,25967,327,56,0.130603,0.349790,-1.666153,-0.324779,0.775324,-0.332835,0.099271,0.122137,-0.2,0.644509,0.691407,-0.613169,-0.515772
1558077,25967,327,57,-0.579598,0.429622,-1.666153,0.319469,0.308861,0.282723,-0.512750,0.012214,-1.6,-0.424133,0.716855,1.628601,0.928389
1558078,25967,327,58,1.278980,1.711134,-1.522820,0.802655,-0.460541,-0.055348,2.405282,0.043511,1.9,0.283960,-0.914914,0.364198,0.211424


In [3]:
sensors = df.iloc[:,3:].columns.tolist()
sensors

['sensor_00',
 'sensor_01',
 'sensor_02',
 'sensor_03',
 'sensor_04',
 'sensor_05',
 'sensor_06',
 'sensor_07',
 'sensor_08',
 'sensor_09',
 'sensor_10',
 'sensor_11',
 'sensor_12']

In [4]:
labels = pd.read_csv('../input/tabular-playground-series-apr-2022/train_labels.csv')
df = pd.merge(df,labels,on='sequence',how='left')

In [5]:
def create_features(df):
    for sensor in sensors:
        df[f'{sensor}_max'] = df.groupby(['sequence','subject'])[sensor].transform('max')
        df[f'{sensor}_min'] = df.groupby(['sequence','subject'])[sensor].transform('min')
        df[f'{sensor}_std'] = df.groupby(['sequence','subject'])[sensor].transform('std')
        df[f'{sensor}_mean'] = df.groupby(['sequence','subject'])[sensor].transform('mean')
        df[f'{sensor}_median'] = df.groupby(['sequence','subject'])[sensor].transform('median')
        df[f'{sensor}_diff'] = df.groupby(['sequence','subject'])[sensor].diff()
        df[f'{sensor}_diff'].fillna(df[f'{sensor}_diff'].median(),inplace=True)
    return df
df = create_features(df)

In [6]:
df

,sequence,subject,step,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,...,sensor_11_std,sensor_11_mean,sensor_11_median,sensor_11_diff,sensor_12_max,sensor_12_min,sensor_12_std,sensor_12_mean,sensor_12_median,sensor_12_diff
0,0,47,0,-0.196291,0.112395,1.000000,0.329204,-1.004660,-0.131638,-0.127505,...,1.257716,0.010391,0.193416,0.001029,6.674766,-50.658994,8.206880,-1.286985,-0.447997,0.000426
1,0,47,1,-0.447450,0.134454,1.000000,-0.658407,0.162495,0.340314,-0.209472,...,1.257716,0.010391,0.193416,-0.763374,6.674766,-50.658994,8.206880,-1.286985,-0.447997,-4.297101
2,0,47,2,0.326893,-0.694328,1.000000,0.330088,0.473678,1.280479,-0.094718,...,1.257716,0.010391,0.193416,-0.354938,6.674766,-50.658994,8.206880,-1.286985,-0.447997,-5.190537
3,0,47,3,0.523184,0.751050,1.000000,0.976991,-0.563287,-0.720269,0.793260,...,1.257716,0.010391,0.193416,1.931070,6.674766,-50.658994,8.206880,-1.286985,-0.447997,5.165388
4,0,47,4,0.272025,1.074580,1.000000,-0.136283,0.398579,0.044877,0.560109,...,1.257716,0.010391,0.193416,-1.221193,6.674766,-50.658994,8.206880,-1.286985,-0.447997,-0.652600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1558075,25967,327,55,-0.282844,-1.217437,-1.666153,0.586726,-0.930698,-0.451010,-0.651184,...,1.144242,-0.009208,0.027778,0.374486,2.654731,-2.193521,1.166639,-0.025163,-0.049659,-1.554561
1558076,25967,327,56,0.130603,0.349790,-1.666153,-0.324779,0.775324,-0.332835,0.099271,...,1.144242,-0.009208,0.027778,-0.259259,2.654731,-2.193521,1.166639,-0.025163,-0.049659,0.398977
1558077,25967,327,57,-0.579598,0.429622,-1.666153,0.319469,0.308861,0.282723,-0.512750,...,1.144242,-0.009208,0.027778,2.241770,2.654731,-2.193521,1.166639,-0.025163,-0.049659,1.444160
1558078,25967,327,58,1.278980,1.711134,-1.522820,0.802655,-0.460541,-0.055348,2.405282,...,1.144242,-0.009208,0.027778,-1.264403,2.654731,-2.193521,1.166639,-0.025163,-0.049659,-0.716965


In [7]:
y = df['state'].copy()
x = df.drop(columns=['state'])

In [8]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(
    x, y, test_size=0.3, random_state=42)

In [9]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate

lgb = LGBMClassifier(random_state=42)
lgb.fit(x_train, y_train)
y_pred = lgb.predict(x_val)

In [10]:
splitter = StratifiedKFold(n_splits = 3, shuffle = True, random_state=42)
scores = cross_validate(lgb, x_train, y_train, return_train_score = True, cv=splitter)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8811513437784232 0.8793670964997213


In [11]:
test = pd.read_csv('../input/tabular-playground-series-apr-2022/test.csv')
test = create_features(test)

In [12]:
pred = lgb.predict(test)
pred

array([1, 1, 1, ..., 0, 0, 0])

In [13]:
submission = pd.read_csv('../input/tabular-playground-series-apr-2022/sample_submission.csv')
submission

,sequence,state
0,25968,0
1,25969,0
2,25970,0
3,25971,0
4,25972,0
...,...,...
12213,38181,0
12214,38182,0
12215,38183,0
12216,38184,0


In [14]:
test['state'] = pred
test

,sequence,subject,step,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,...,sensor_11_mean,sensor_11_median,sensor_11_diff,sensor_12_max,sensor_12_min,sensor_12_std,sensor_12_mean,sensor_12_median,sensor_12_diff,state
0,25968,684,0,2.427357,19.639706,1.00000,-1.466372,-1.289973,-4.207928,2.486339,...,0.025977,0.121399,0.001029,0.716539,-0.809889,0.369897,-0.000945,-0.008951,-0.000853,1
1,25968,684,1,-4.950541,-21.747899,1.00000,0.983186,-0.569053,1.845924,-3.887978,...,0.025977,0.121399,-12.851852,0.716539,-0.809889,0.369897,-0.000945,-0.008951,-0.427536,1
2,25968,684,2,1.136012,-10.756303,1.00000,1.016814,0.964157,2.454749,0.312386,...,0.025977,0.121399,-9.712963,0.716539,-0.809889,0.369897,-0.000945,-0.008951,0.148338,1
3,25968,684,3,0.806028,6.504202,1.00000,-0.179646,0.969221,-1.035153,-0.457195,...,0.025977,0.121399,7.342593,0.716539,-0.809889,0.369897,-0.000945,-0.008951,-0.602728,1
4,25968,684,4,1.288253,5.552521,1.00000,-0.493805,-1.036124,-1.126402,2.008197,...,0.025977,0.121399,3.769547,0.716539,-0.809889,0.369897,-0.000945,-0.008951,0.341432,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733075,38185,773,55,0.211747,2.005252,-1.33282,0.695575,-0.161327,-1.193717,0.421676,...,-0.047308,0.121914,3.027778,207.146633,-161.786445,92.680510,1.558234,20.058397,-117.942029,0
733076,38185,773,56,-0.826121,-2.468487,-1.33282,0.381416,0.144745,1.060583,-0.765938,...,-0.047308,0.121914,-4.000000,207.146633,-161.786445,92.680510,1.558234,20.058397,164.025575,0
733077,38185,773,57,0.755023,1.469538,-1.33282,-1.253097,-0.414802,0.007479,0.907104,...,-0.047308,0.121914,2.436214,207.146633,-161.786445,92.680510,1.558234,20.058397,-106.480392,0
733078,38185,773,58,-0.187017,0.714286,-1.33282,0.077876,1.323245,0.159312,-0.397996,...,-0.047308,0.121914,-0.324074,207.146633,-161.786445,92.680510,1.558234,20.058397,99.725064,0


In [15]:
t = test.groupby('sequence').mean().reset_index()
t.loc[t['state']<0.5,'state'] = 0
t.loc[t['state']>=0.5,'state'] = 1
# submission['state'] = t['state']
# submission

In [16]:
submission['state'] = t['state']
submission.to_csv('submission.csv',index=False)

In [17]:
# scaler = StandardScaler()
# scaler.fit(df.iloc[:,3:])
# df.iloc[:,3:] = scaler.transform(df.iloc[:,3:])
# df

In [18]:
# fig, ax = plt.subplots(13,figsize=(15,25))
# for i in range(0,13):
#     ax[i].plot(a['sensor_'+f"{i:02d}"])

In [19]:
# a.iloc[:,3:] = a.mean().tolist()[3:]
# a

In [20]:
# fig, ax = plt.subplots(13,figsize=(15,25),sharey=True,sharex=True)
# for i in range(0,13):
#     ax[i].plot(a['sensor_'+f"{i:02d}"])

In [21]:
df.describe()

,sequence,subject,step,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,...,sensor_11_std,sensor_11_mean,sensor_11_median,sensor_11_diff,sensor_12_max,sensor_12_min,sensor_12_std,sensor_12_mean,sensor_12_median,sensor_12_diff
count,1.558080e+06,1.558080e+06,1.558080e+06,1.558080e+06,1.558080e+06,1.558080e+06,1.558080e+06,1.558080e+06,1.558080e+06,1.558080e+06,...,1.558080e+06,1.558080e+06,1.558080e+06,1.558080e+06,1.558080e+06,1.558080e+06,1.558080e+06,1.558080e+06,1.558080e+06,1.558080e+06
mean,1.298350e+04,3.316331e+02,2.950000e+01,4.365526e-04,-1.034982e-03,-2.178045e-01,-2.156555e-03,-1.828903e-03,-1.651785e-03,-4.122917e-04,...,1.905523e+00,-3.733291e-03,5.646848e-03,-7.731500e-06,3.773305e+01,-3.749747e+01,1.783660e+01,-1.172605e-02,-9.810661e-02,-1.344679e-03
std,7.496318e+03,1.958257e+02,1.731811e+01,2.658684e+00,4.404200e+00,2.298002e+00,3.934184e+00,1.683685e+00,1.590818e+00,3.345143e+00,...,4.152142e+00,1.439652e-01,2.057055e-01,7.090854e+00,7.179160e+01,7.105486e+01,3.514747e+01,1.610842e+00,4.703744e+00,4.743822e+01
min,0.000000e+00,0.000000e+00,0.000000e+00,-3.750634e+02,-4.345977e+02,-3.165948e+01,-4.083761e+02,-2.362601e+01,-7.498280e+01,-4.705046e+02,...,0.000000e+00,-6.418759e+00,-1.262603e+01,-7.680720e+02,0.000000e+00,-6.125494e+02,0.000000e+00,-5.133205e+01,-9.479049e+01,-7.839190e+02
25%,6.491750e+03,1.617500e+02,1.475000e+01,-5.000000e-01,-4.831933e-01,-6.461531e-01,-4.929204e-01,-4.729928e-01,-4.786836e-01,-4.927140e-01,...,6.238963e-01,-1.529492e-02,-4.166667e-02,-7.983539e-01,6.520673e-01,-4.946260e+01,2.796321e-01,-1.867363e-02,-8.061594e-02,-8.107417e-01
50%,1.298350e+04,3.350000e+02,2.950000e+01,-3.091190e-03,3.151261e-03,0.000000e+00,0.000000e+00,-1.589577e-03,2.991773e-03,9.107468e-04,...,9.253914e-01,-2.117627e-03,4.115226e-03,1.028807e-03,2.135763e+00,-2.084186e+00,9.046389e-01,-2.841716e-05,-2.131287e-04,4.262575e-04
75%,1.947525e+04,5.010000e+02,4.425000e+01,4.845440e-01,4.926471e-01,3.338469e-01,4.893805e-01,4.701565e-01,5.056096e-01,4.927140e-01,...,1.608490e+00,1.056670e-02,5.349794e-02,8.034979e-01,4.951535e+01,-6.602728e-01,2.290551e+01,1.815679e-02,6.649616e-02,8.115942e-01
max,2.596700e+04,6.710000e+02,5.900000e+01,3.358246e+02,4.495914e+02,1.666667e+00,4.366504e+02,2.487286e+01,7.791548e+01,4.425009e+02,...,1.456417e+02,3.149640e+00,9.819444e+00,6.938447e+02,6.305111e+02,0.000000e+00,3.476631e+02,2.862626e+01,1.072159e+02,8.943662e+02


In [22]:
# def signal(d):
#     for i in range(0,13):
#         d.loc[(d['sensor_'+f"{i:02d}"]>-0.1) & (d['sensor_'+f"{i:02d}"]<0.1),'sensor_'+f"{i:02d}"]=0
#         d.loc[(d['sensor_'+f"{i:02d}"]!=0),'sensor_'+f"{i:02d}"]=1
#     d = d.astype('int64')
#     return d
# signal(df)

In [23]:
# def signal(d):
#     a=''
#     for i in range(0,13):
#         a = '0'+str(i)
#         if(i>=10):
#             a=str(i)
#         d.loc[(d['sensor_'+a]>-2) & (d['sensor_'+a]<2),'sensor_'+a]=0
#         d.loc[(d['sensor_'+a]!=1),'sensor_'+a]=1
#     d = d.astype('int64')
#     return d
# df = signal(df)
# test = signal(test)

In [24]:
# a = df.groupby('subject').mean()

In [25]:
# a[a.iloc[:,2:15]>0.5] = 1
# a[a.iloc[:,2:15]<=0.5] = 0
# a = a.astype('int')
# a

In [26]:
# plt.figure(figsize=(25,13))
# sns.scatterplot(data=df.groupby(['sequence']).first(),x='sequence',y='sensor_00',hue='state')

In [27]:
# df.drop(columns=['step','sequence'],inplace=True)
# df = df.astype('int')
# df

In [28]:
# df.drop(columns=['step','sequence','subject'],inplace=True)
# test.drop(columns=['step','sequence','subject'],inplace=True)
# sns.heatmap(df.corr())

In [29]:
# sns.histplot(df.drop_duplicates()['sequence'].value_counts())
# agg(lambda x:x.value_counts().index[0])


In [30]:
# df = df.groupby(['sequence','subject']).agg(lambda x:x.value_counts().index[0])
# df = df.reset_index()
# df

In [31]:
# df['sensor_03'].value_counts()

In [32]:
# sns.heatmap(df.corr(),vmax=0.6,vmin=0.4)
# df[(df['sensor_00']==1) & (df['sensor_01']==1)]
# df[df.iloc[:,3:]==1].sum()
# df[(df['sensor_00']==1) & (df['sensor_01']==1) & (df['sensor_02']==1) & (df['sensor_03']==1) & (df['sensor_04']==1) & (df['sensor_05']==1) & (df['sensor_06']==1) & (df['sensor_07']==1) & (df['sensor_08']==1) & (df['sensor_09']==1) & (df['sensor_10']==1) & (df['sensor_11']==1) & (df['sensor_12']==1)]

In [33]:
# fig,ax = plt.subplots()
# sns.scatterplot(data=df,x='sensor_00',y='sensor_01')
# sns.scatterplot(data=df.reset_index(),x='index',y='sensor_01')
# sns.scatterplot(data=df.reset_index(),x='index',y='sensor_02')

In [34]:
# train_split = int(0.715*int(df.shape[0]))
# train_data = df.loc[0:train_split-1]
# val_data = df.loc[train_split:]
# x_train = train_data.drop(columns=['state']).values
# y_train = train_data['state'].values

In [35]:
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.wrappers.scikit_learn import KerasClassifier
# from sklearn.model_selection import cross_val_score
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import StratifiedKFold
# from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import Pipeline
# encoder = LabelEncoder()
# encoder.fit(y_train)
# encoded_Y = encoder.transform(y_train)
# def create_larger():
#     model = Sequential()
#     model.add(Dense(13, input_dim=13, activation='relu'))
#     model.add(Dense(90, activation='relu'))
#     model.add(Dense(30, activation='relu'))
#     model.add(Dense(15, activation='relu'))
#     model.add(Dense(1, activation='sigmoid'))
#     model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#     return model

# estimators = []
# # estimators.append(('standardize', StandardScaler()))
# estimators.append(('mlp', KerasClassifier(build_fn=create_larger, epochs=15, batch_size=500, verbose=0)))
# pipeline = Pipeline(estimators)
# kfold = StratifiedKFold(n_splits=10, shuffle=True)
# results = cross_val_score(pipeline, x_train, y_train, cv=kfold)
# print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [36]:
# train_data[0:60].groupby(train_data.columns.tolist(),as_index=False).size()

In [37]:
# print(x_train.shape)
# print(x_train.shape)
# sequence_length = int(len(df)/60)
# sampling_rate = 60
# batch_size = 256
# sequence_length

In [38]:
# import tensorflow as tf
# dataset_train = tf.keras.preprocessing.timeseries_dataset_from_array(
#     x_train,
#     y_train,
#     sequence_length=100,
#     sampling_rate=60,
#     batch_size=256,
# )
# dataset_train

In [39]:
# import tensorflow as tf
# dataset = tf.data.Dataset.from_tensor_slices(x_train)
# print(list(dataset.as_numpy_iterator()))

In [40]:
# dataset_val = tf.keras.preprocessing.timeseries_dataset_from_array(
#     val_data.drop(columns=['state']),
#     val_data['state'],
#     sequence_length=100,
#     sampling_rate=60,
#     batch_size=256,
# )

In [41]:
# for batch in dataset_train.take(1):
#     inputs, targets = batch

In [42]:
# X_train = pd.concat([X_train,X_encode],axis=0)
# X_train
# from tensorflow import keras
# print("Input shape:", inputs.numpy().shape)
# print("Target shape:", targets.numpy().shape)

In [43]:
# inputs = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]))
# lstm_out = keras.layers.LSTM(32)(inputs)
# outputs = keras.layers.Dense(1,activation='sigmoid')(lstm_out)

# model = keras.Model(inputs=inputs, outputs=outputs)
# model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="binary_crossentropy")
# model.summary()

In [44]:
# path_checkpoint = "model_checkpoint.h5"
# es_callback = keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=5)

# modelckpt_callback = keras.callbacks.ModelCheckpoint(
#     monitor="val_loss",
#     filepath=path_checkpoint,
#     verbose=1,
#     save_weights_only=True,
#     save_best_only=True,
# )

# history = model.fit(
#     dataset_train,
#     epochs=10,
#     validation_data=dataset_val,
#     callbacks=[es_callback, modelckpt_callback],
# )

In [45]:
# X_train.sort_index(inplace=True)

In [46]:
# X_train = X_train.drop(columns=['sequence'])
# X_train
# df['sensor_09'].value_counts()

In [47]:
# from sklearn.model_selection import train_test_split
# train_X, test_X, train_Y, test_Y = train_test_split(df.drop(columns=['state']),df['state'],test_size=0.33,random_state=42)

In [48]:
# from sklearn.ensemble import RandomForestClassifier
# rfc = RandomForestClassifier(max_depth=10, random_state=42)
# rfc.fit(train_X,train_Y)

In [49]:
# pred_rfc = rfc.predict(test_X)
# pred_rfc

In [50]:
# from sklearn.metrics import roc_auc_score
# roc_auc_score(pred_rfc,test_Y)

In [51]:
# from xgboost import XGBClassifier
# xgb = XGBClassifier(max_depth=13,random_state=42,eval_metric='auc',use_label_encoder=False)
# xgb.fit(train_X,train_Y)

In [52]:
# roc_auc_score(test_Y,xgb.predict(test_X))

In [53]:
# df_train = train_X.sample(frac=0.7, random_state=0)
# df_valid = train_X.drop(df_train.index)
# max_ = df_train['step'].max(axis=0)
# min_ = df_train['step'].min(axis=0)
# df_train['step'] = (df_train['step'] - min_) / (max_ - min_)
# df_valid['step'] = (df_valid['step'] - min_) / (max_ - min_)

In [54]:
# y_train = Y[df_train.index]
# y_valid = Y[df_valid.index]

In [55]:
# from tensorflow import keras
# from tensorflow.keras import layers

# model = keras.Sequential([
#     layers.Dense(4, activation='relu', input_shape=[16]),
#     layers.Dense(4, activation='relu'),    
#     layers.Dense(1, activation='sigmoid'),
# ])

In [56]:
# model.compile(
#     optimizer='adam',
#     loss='binary_crossentropy',
#     metrics=['binary_accuracy'],
# )
# early_stopping = keras.callbacks.EarlyStopping(
#     patience=10,
#     min_delta=0.001,
#     restore_best_weights=True,
# )

# history = model.fit(
#     df_train, y_train,
#     validation_data=(df_valid, y_valid),
#     batch_size=512,
#     epochs=1000,
#     callbacks=[early_stopping],
#     verbose=0,
# )